In [1]:
import torch
import torch.nn.functional as F
import numpy as np

from ssgd import StreamingSGD

In [3]:
torch.set_printoptions(precision=10)

In [4]:
torch.manual_seed(9)

# Model definition

A StreamingSGD compatible model now needs to be able to "detach" layers as well as gather input/output and gradients. It also needs a list of layers. See below for implementation example. In the future we want to implement this using PyTorch hooks and modules.

In [5]:
class ExampleNet(torch.nn.Module):
    def __init__(self):
        super(ExampleNet, self).__init__()
        
        self.input_layer = torch.nn.Conv2d(3, 3, kernel_size=3)
        self.layers = [self.input_layer]

        for i in range(5):  # use 9 for 8194 x 8194 images
            self.add_block(i)
        
        final_conv5 = torch.nn.Conv2d(3, 1, kernel_size=10)
        self.add_module("final", final_conv5)
        
        self.layers.extend([final_conv5])
        
    def add_block(self, i):
        conv1 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv2 = torch.nn.Conv2d(3, 3, kernel_size=3)
        conv3 = torch.nn.Conv2d(3, 3, kernel_size=3)
        maxpool = torch.nn.MaxPool2d(2, stride=2)
        
        self.add_module("conv1-" + str(i), conv1)
        self.add_module("conv2-" + str(i), conv2)
        self.add_module("conv3-" + str(i), conv3)
        self.add_module("maxpool-" + str(i), maxpool)

        self.layers.extend([conv1, conv2, conv3, maxpool])

    def forward(self, x, stop_at_layer=None, start_at_layer=None):
        stop_index, start_index = -1, 0
        for i, (name, layer) in enumerate(self.named_modules()):
            if name == stop_at_layer:
                stop_index = i
            if name == start_at_layer:
                start_index = i
        
        for i, layer in enumerate(self.layers[start_index:]):
            if i == stop_index:
                break
                
            if i + 1 == len(self.layers[start_index:]):
                x = layer(x)
                x = x.view(-1, 1)
                x = F.sigmoid(x)
            else:
                x = F.relu(layer(x))

        return x

model = ExampleNet()
# model = model.double()

Weight initialization; we use positive values to generate large gradients, better for testing if final gradients are correct.

In [6]:
for i, layer in enumerate(model.modules()):
    if isinstance(layer, torch.nn.Conv2d):
        layer.weight.data *= 2.5
        layer.bias.data.zero_()

In [7]:
print(model)

ExampleNet(
  (input_layer): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv1-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-1): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3-2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (maxpool-2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1-3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2-3): Conv2d(3, 3, kernel_size=(3, 3

# Configurations

In [8]:
stop_layer = 'conv1-3'  # use 'conv3-4' for 8194x8194 images
img_size = 514  # try 8194, see last segment for details

cuda = False # execute this notebook on the GPU
verbose = True  # enable / disable logging
divide_in = 8  # tip: use 25 for 8194x8194 when memory constraint

# Configure streaming SGD

In [9]:
if cuda:
    model.cuda()

In [10]:
sCNN = StreamingSGD(model, stream_to_layer=stop_layer, 
               input_shape=(1, 3, img_size, img_size), 
               divide_in=divide_in, 
               cuda=cuda, 
               verbose=verbose)

Feature map to be reconstructed shape: (56.0, 56.0)
Feature map divided in tile sizes: (7.0, 7.0)
Tile size forward: (tensor(122.), tensor(122.))
Tile size backward (for forward pass): (tensor(186.), tensor(186.))
*** Approximate memory reduction of streaming: 86.9% ***



# Generate random image and fake label

In the current implementation the whole image needs to be able to fit in memory (RAM).

In [11]:
image = torch.FloatTensor(3, img_size, img_size).normal_(0, 1)
target = torch.FloatTensor(1, 1).fill_(0)

# image = image.double()
# target = target.double()

if cuda:
    target = target.cuda()
    image = image.cuda()

In [12]:
criterion = torch.nn.BCELoss()

# Run through network using streaming

In [13]:
output, feature_map = sCNN.forward(image); output

  0%|          | 0/64 [00:00<?, ?it/s]

Doing forward pass...


100%|██████████| 64/64 [00:00<00:00, 508.42it/s]


tensor([[0.5723423362]])

In [14]:
loss = criterion(output, target); loss

tensor(0.8494322300)

In [15]:
full_gradients = sCNN.backward(image, feature_map, loss)

  5%|▍         | 3/64 [00:00<00:02, 23.18it/s]

Doing backward pass...


100%|██████████| 64/64 [00:02<00:00, 29.68it/s]

Everything reconstructed:
 True


"Everything reconstructed" means that all gradients were reconstructed succesfully!

Save the gradients of the conv2d layer to compare with normal SGD:

In [16]:
streaming_conv_gradients = []

for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            streaming_conv_gradients.append(layer.weight.grad.clone()) 

# Compare to normal SGD

Reset the gradients and perform a normal for backward pass.

In [17]:
for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            layer.weight.grad.data.zero_()
            layer.bias.grad.data.zero_()

This output should be the same as the streaming SGD output, if so the loss will also be the same:

In [18]:
output = model(image[None]); output

tensor([[0.5723423362]])

In [19]:
loss = criterion(output, target); loss

tensor(0.8494322300)

In [20]:
loss.backward()

# Compare the gradients of the conv2d layers

Save the gradients of the conv2d layer to compare with normal SGD:

In [21]:
normal_conv_gradients = []
j = 0
for i, layer in enumerate(model.layers):
    if isinstance(layer, torch.nn.Conv2d):
        if layer.weight.grad is not None:
            normal_conv_gradients.append(layer.weight.grad) 
            j += 1

In [22]:
for i in range(len(streaming_conv_gradients)):
    print("Conv layer", i, "\taverage gradient size:", float(torch.mean(torch.abs(streaming_conv_gradients[i].data))))

Conv layer 0 	average gradient size: 0.8599761128425598
Conv layer 1 	average gradient size: 0.9677515625953674
Conv layer 2 	average gradient size: 0.45716390013694763
Conv layer 3 	average gradient size: 0.45249807834625244
Conv layer 4 	average gradient size: 0.3712787628173828
Conv layer 5 	average gradient size: 0.4326077699661255
Conv layer 6 	average gradient size: 0.2890121340751648
Conv layer 7 	average gradient size: 0.4207112193107605
Conv layer 8 	average gradient size: 0.49478083848953247
Conv layer 9 	average gradient size: 0.31336453557014465
Conv layer 10 	average gradient size: 0.20127855241298676
Conv layer 11 	average gradient size: 0.17118842899799347
Conv layer 12 	average gradient size: 0.0870930477976799
Conv layer 13 	average gradient size: 0.043339017778635025
Conv layer 14 	average gradient size: 0.04104870930314064
Conv layer 15 	average gradient size: 0.041901979595422745
Conv layer 16 	average gradient size: 0.14884939789772034


In [23]:
for i in range(len(streaming_conv_gradients)):
    max_diff = torch.max(torch.abs(streaming_conv_gradients[i].data - 
                         normal_conv_gradients[i].data))
    print("Conv layer", i, "\tmax difference between gradients:", float(max_diff))

Conv layer 0 	max difference between gradients: 2.86102294921875e-06
Conv layer 1 	max difference between gradients: 2.384185791015625e-06
Conv layer 2 	max difference between gradients: 1.6689300537109375e-06
Conv layer 3 	max difference between gradients: 1.7881393432617188e-06
Conv layer 4 	max difference between gradients: 8.344650268554688e-07
Conv layer 5 	max difference between gradients: 9.5367431640625e-07
Conv layer 6 	max difference between gradients: 5.364418029785156e-07
Conv layer 7 	max difference between gradients: 1.0728836059570312e-06
Conv layer 8 	max difference between gradients: 7.748603820800781e-07
Conv layer 9 	max difference between gradients: 2.384185791015625e-07
Conv layer 10 	max difference between gradients: 5.066394805908203e-07
Conv layer 11 	max difference between gradients: 0.0
Conv layer 12 	max difference between gradients: 0.0
Conv layer 13 	max difference between gradients: 0.0
Conv layer 14 	max difference between gradients: 0.0
Conv layer 15 	ma

As you can see the difference of the gradients of the conv2d layers between the methods is (almost) numerically equivalent. The small differences are because of loss of significance with the floating points calculations. 

---

# Things to try:

* Use doubles instead of floats to reduce the difference (use model.double() and image_var.double())
* Make the image bigger than would fit on a GPU 
    - e.g. 8194x8194, make sure to add 3 more blocks in the model (see comments)
* If you want you can compare the reconstructed input gradients of each layer: 
    - pass fill_gradient=True in backward() function
    - compare full_gradients with self.model.gradients after the full model backward pass.
* For testing purposes the number of filters is small in this notebook, try increasing them